In [1]:
import time 
import warnings 

import numpy as np
import pandas as pd

from sklearn import cluster
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import StandardScaler

import plotly.plotly as py
import plotly.graph_objs as go

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [14]:
from sklearn import metrics as metrics

In [57]:
def metricPlot(coords,dim):
    marker=dict(
        size=5,
        line=dict(
            color='rgb(0, 0, 0)',
            width=0.5
        )
    )
    
    if dim==3:
        tmp=pd.DataFrame(coords, columns=['para1','para2','coefficient','time','cluster_num'])
        X=tmp['para1']
        Y=tmp['para2']
        Z=tmp['coefficient']
        #text="time: "+ str(tmp['time'])+", clusters: "+ str(tmp['cluster_num'])
        
        trace = go.Scatter3d(
            x=X,
            y=Y,
            z=Z,
            text=tmp['cluster_num'],
            mode='markers',
            marker=marker
        )
    else:
        tmp=pd.DataFrame(coords, columns=['para1','coefficient','time'])
        X=tmp['para1']
        Y=tmp['coefficient']
        text=tmp['time']
        
        trace = go.Scatter(
            x=X,
            y=Y,
            text=text,
            mode='markers',
            marker=marker
        )         

    data = [trace]
    
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene = dict(
        xaxis=dict(title='eps'),
        yaxis=dict(title='min_samples'),
        zaxis=dict(title='silhouette')
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, world_readable=True, filename='specStudyModel')

In [6]:
def clustering(userModel):

    #X=userModel.loc[userModel['theme_id']=='27688ee203c144bb83cff0cceab9a483']
    X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration']]    
    
    X = StandardScaler().fit_transform(X)
    
    floatArr=[i for i in np.arange(0.1,1.1,0.1)]
    intArr=[i for i in range(2,10)]
    
    #eps float and minsamples int
    #algorithm=cluster.DBSCAN()   
    
    #eigen_tol float and n_clusters int (optional)
    #algorithm=cluster.SpectralClustering() #Поиграть с параметрами
    
    #n_clusters int
    #algorithm=cluster.AgglomerativeClustering()
    
    #n_clusters int and batch_size int
    algorithm=cluster.MiniBatchKMeans()
    
    #dim=3
    dim=2
    
    coords=[]
    
    #for para1 in floatArr:
    for para1 in intArr:
        for para2 in intArr:
            davies_bouldin,silhouette,time=clusteringParams(X,para1,para2,algorithm)
            
            coords.append((para1,para2,silhouette,davies_bouldin,time))
    """
    for para1 in intArr:
        silhouette,time=clusteringParams(X,para1,0,algorithm)
        coords.append((para1,silhouette,time))
    """
        
            
    return coords,dim
    

In [2]:
def clusteringParams(X,para1,para2,algorithm):

    #algorithm.set_params(eps=para1, min_samples=para2) #dbscan
    #algorithm.set_params(n_clusters=para2,eigen_tol=para1) #spectural
    #algorithm.set_params(n_clusters=para1) #agglomerative
    #algorithm.set_params(n_clusters=para1,batch_size=para2) #k-means
    
    t0 = time.time()
    
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="the number of connected components of the " +
            "connectivity matrix is [0-9]{1,2}" +
            " > 1. Completing it to avoid stopping the tree early.",
            category=UserWarning)
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding" +
            " may not work as expected.",
            category=UserWarning)
        algorithm.fit(X)
    t1 = time.time()
    
    labels_num=len(np.unique(algorithm.labels_))

    if len(np.unique(algorithm.labels_))>1:
        silhouette=silhouette_score(X,labels=algorithm.labels_)
        
    else:
        silhouette=-1
       
    return silhouette, (t1-t0) #, labels_num
    
    
    

In [8]:
#from getUserStudyModel import getModel
#userModel=getModel()

userModel = pd.read_csv('problems_starts_time.csv', sep=',', encoding='utf-8')

In [2]:
#userModel.to_csv(path_or_buf='problems_starts_time.csv',index=False)

In [16]:
coords,dim=clustering(userModel)

AttributeError: module 'sklearn.metrics' has no attribute 'davies_bouldin_score'

In [ ]:
tmp=pd.DataFrame(coords, columns=['para1','para2','silhouette','time'])

In [ ]:
maxCoef=max(list(tmp['silhouette']))
maxCoef

In [ ]:
t=tmp.loc[tmp['silhouette']==minCoef]

In [ ]:
t

In [52]:
metricPlot(coords,dim)

In [ ]:
#скорость работы

In [44]:
userModel = pd.read_csv('problems_starts_time.csv', sep=',', encoding='utf-8')
#problems_starts_time
X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration']]    
X = StandardScaler().fit_transform(X)

In [ ]:
algorithm1=cluster.DBSCAN(eps=1.0, min_samples=3)
algorithm2=cluster.SpectralClustering(n_clusters=3,eigen_tol=0.9)
algorithm3=cluster.AgglomerativeClustering(n_clusters=2)
algorithm4=cluster.MiniBatchKMeans(n_clusters=2,batch_size=5)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4]

In [46]:
algotime=[]
pointsX=[]
for algorithm in algorithms:
    tmpAlgoTime=[]
    for dataPercent in range(10,101,10):
        size=round(X.shape[0]*dataPercent/100)
        if len(pointsX)<10:
            pointsX.append(size)
        tmpX=X[0:size]
        silhouette, wtime = clusteringParams(tmpX,0,0,algorithm)
        tmpAlgoTime.append(wtime)
    algotime.append(tmpAlgoTime)

In [47]:
df=pd.DataFrame(algotime).transpose()

In [49]:
names = ['DBSCAN','Spectral Clustering','Agglomerative clustering','K-means']

data=[]
for algoNum in range(df.shape[1]):
    Y=df[algoNum]
    data.append(
        go.Scatter(
            x=pointsX,
            y=Y,
            mode='markers+lines',
            marker=dict(size=5),
            name=names[algoNum]
        )   
    )


layout = go.Layout(        
        xaxis=dict(title='Количество данных для обучения',ticklen= 5, showticklabels=True),
        yaxis=dict(title='Время, потраченное на обучение (в секундах)',ticklen= 5,zeroline= True,showticklabels=True),
    legend=dict(x=0.01, y=1)
    
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, world_readable=True, filename='specStudyModel')

In [ ]:
#основные кластеры

In [ ]:
algorithm1=cluster.DBSCAN(eps=1.0, min_samples=3)
algorithm2=cluster.SpectralClustering(n_clusters=3,eigen_tol=0.9)
algorithm3=cluster.AgglomerativeClustering(n_clusters=2)
algorithm4=cluster.MiniBatchKMeans(n_clusters=2,batch_size=5)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4]

In [116]:
#userModel = pd.read_csv('problems_starts_time.csv', sep=',', encoding='utf-8')
#problems_starts_time
X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration']]    
scal = StandardScaler()
X_t = scal.fit_transform(X)


algorithm1.fit(X_t)
#centers=scal.inverse_transform(algorithm3.cluster_centers_)

DBSCAN(algorithm='auto', eps=1.0, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=3, n_jobs=1, p=None)

In [117]:
centers=scal.inverse_transform(algorithm4.cluster_centers_)
centers

array([[2.95137619e+03, 3.85443615e+03, 1.29870130e-02, 2.05130717e+02],
       [4.54747351e-13, 0.00000000e+00, 1.73472348e-18, 7.67074651e+04]])

In [118]:
np.unique(algorithm1.labels_)

array([-1,  0,  1,  2], dtype=int64)

In [119]:
tmp=pd.DataFrame(algorithm1.labels_)

In [120]:
labeledModel=userModel.join(tmp)

In [130]:
forcalc=labeledModel.loc[labeledModel[0]==2]
calc=forcalc.groupby('course_id')['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration'].mean()

In [131]:
calc

,problem_duration_graded,problem_duration_ungraded,messagiesNum,video_duration
course_id,,,,
course-v1:spbu+SOCLAB+spring_2018,0.0,0.0,3.0,0.0


In [132]:
forcalc.shape

(3, 8)